In [1]:
import torch
from torch import optim
import torch.nn as nn
import numpy as np
import time

import Model_inputs as Mi
import Model_definition as Md

Device: cuda
Device: cuda
Device: cuda


In [2]:
if torch.cuda.is_available():
    my_device = torch.device('cuda')
else:
    my_device = torch.device('cpu')
print('Device: {}'.format(my_device))

Device: cuda


In [3]:
#import the model
SOAP_predictor = Md.SOAP_predictor_model().to(my_device)

In [4]:
#define optimiser and loss
optimiser = optim.Adam(SOAP_predictor.parameters(), lr = 5*10**(-5))
mean_sq_loss = nn.MSELoss(reduction="mean")

In [ ]:
start_epochs = time.time()

epochs = 1000

B = 100 #batch size
N = 10 #molecle size
alpha = 0.3 #controls the amount of white noise
no_teacher_forcing_prob = 0.7 

#extent of AFM image
lenght_width = 10
height = 2

optimiser.zero_grad()

#save loss:
#rows: number of nodes in graph
#columns: epochs
loss_lists = [[],[],[],[],[],[],[],[],[],[]]
mean_loss_list = [] #over all nodes

for e in range(epochs):
    print("\n", "\n")
    print("Epoch: " + str(e))
    
    print("Generate batch of molecules")
    with torch.no_grad():
        #create input
        #atom class buckets
        coordinate_buckets = Mi.create_buckets(lenght_width, height, B, N)[2]
        number_classes = len(coordinate_buckets)
        #generate input order
        history, molecule_input, molecule_string, number_per_class = Mi.create_input_order(number_classes, coordinate_buckets, B, N)
        #re-order points by Euclidean distance
        molecule_input = Mi.Euclid_reorder(molecule_input)
    
    print("Predict SOAP for all subgraphs from 1 to " + str(N) + " atoms per subgraph")
    #feed atoms one by one to the net
    #model prediction on intermediate graph
    #calculate label and loss
    #BP
    SOAP_n_1 = None
    chem_env_n_1 = []
    for n in range(1, N+1):
        
        print("Number nodes: " + str(n))
        print("Create subgraph input and label")
        with torch.no_grad():
            #net inputs
            input_graph_enc, input_node_centre_enc, input_SOAP_pred, chem_env_n = Mi.make_network_inputs(molecule_input, molecule_string, SOAP_n_1 , chem_env_n_1, n, B)
            sorted_species_list = Mi.env_manager(set(molecule_string[0:n])).species
            #create label
            SOAP_label = Mi.make_label(n, molecule_string, molecule_input, B)[0]
        
        print("Model prediction")
        
        st = time.time()
        SOAP_prediction = SOAP_predictor(input_graph_enc, input_node_centre_enc, input_SOAP_pred, chem_env_n, chem_env_n_1, sorted_species_list, SOAP_n_1, n)
        et = time.time()
        print("Elapsed time: ", et-st)
        
        loss = mean_sq_loss(SOAP_prediction, SOAP_label)
        print("Loss: ", loss.item())
        
        print("start BP")
        st = time.time()
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()
        et = time.time()
        print("end BP")
        print("Elapsed time: ", et-st)
        
        with torch.no_grad():
            #update loss
            loss_lists[n-1].append(loss.item())
            #prepare inputs from previous iteration for next one
            SOAP_n_1 = SOAP_prediction.detach()
            chem_env_n_1 = chem_env_n
            #apply teacher forcing according to its probability
            if np.random.uniform(0, 1) > no_teacher_forcing_prob:
                SOAP_n_1 = Mi.make_label(n, molecule_string, molecule_input, B)[0]
                noise = torch.randn(SOAP_n_1.size(), requires_grad = False, dtype = torch.float, device=my_device)*alpha
                SOAP_n_1 = SOAP_n_1 + noise*SOAP_n_1
                
        
    with torch.no_grad():
        #compute mean loss
        mean_loss = np.sum(np.array(loss_lists)[:, -1])/N
        mean_loss_list.append(mean_loss)
        print("\n")
        print("Mean Loss Epoch {}: ".format(e), mean_loss)
        #save loss and checkpoint model weights
        if (e+1) % 50 == 0:
            np.savetxt("loss.txt", np.array(loss_lists))
            save_model_name = "SOAP_predictor_weights"+ str(e) + ".pth"
            torch.save(SOAP_predictor.state_dict(), save_model_name)
            np.savetxt("mean_loss.txt", np.array(mean_loss_list))
            
end_epochs = time.time()
time_epochs = end_epochs-start_epochs
print("Time for {} epochs: ".format(epochs), time_epochs)


 

Epoch: 0
Generate batch of molecules
Classes allocation : (1, 8, 1)
Class with zero atoms:  None
Final counters:  [1, 8, 1]
Predict SOAP for all subgraphs from 1 to 10 atoms per subgraph
Number nodes: 1
Create subgraph input and label
[('C', 'C')]
Model prediction
Elapsed time:  0.2700819969177246
Loss:  38.23497009277344
start BP
end BP
Elapsed time:  0.024508237838745117
Number nodes: 2
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Model prediction
Elapsed time:  0.02776193618774414
Loss:  14.635031700134277
start BP
end BP
Elapsed time:  0.049019813537597656
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Number nodes: 3
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.04480743408203125
Loss:  5.445985794067383
start BP
end BP
Elapsed time:  0.07226228713989258
Number nodes: 4
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')

end BP
Elapsed time:  0.039285898208618164
[('O', 'O')]
Number nodes: 2
Create subgraph input and label
[('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.020116329193115234
Loss:  14.9866361618042
start BP
end BP
Elapsed time:  0.04026460647583008
Number nodes: 3
Create subgraph input and label
[('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.02736043930053711
Loss:  25.50722312927246
start BP
end BP
Elapsed time:  0.0611569881439209
Number nodes: 4
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.04501938819885254
Loss:  13.889528274536133
start BP
end BP
Elapsed time:  0.09604525566101074
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Number nodes: 5
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.05613422393798828
Loss:  3.78745269775390

Model prediction
Elapsed time:  0.023248910903930664
Loss:  37.98705291748047
start BP
end BP
Elapsed time:  0.036901235580444336
[('C', 'C')]
Number nodes: 2
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Model prediction
Elapsed time:  0.020097017288208008
Loss:  14.906970977783203
start BP
end BP
Elapsed time:  0.041274070739746094
Number nodes: 3
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Model prediction
Elapsed time:  0.029320240020751953
Loss:  25.012042999267578
start BP
end BP
Elapsed time:  0.061287879943847656
Number nodes: 4
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Model prediction
Elapsed time:  0.030138254165649414
Loss:  33.684837341308594
start BP
end BP
Elapsed time:  0.08826756477355957
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Number nodes: 5
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Model prediction
Elapsed time:  0.03262209892272949
Loss:  5.370832443237305
start BP
en

end BP
Elapsed time:  0.09904289245605469
Number nodes: 5
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Model prediction
Elapsed time:  0.03242850303649902
Loss:  32.36125564575195
start BP
end BP
Elapsed time:  0.14570355415344238
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Number nodes: 6
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Model prediction
Elapsed time:  0.03670477867126465
Loss:  5.684132099151611
start BP
end BP
Elapsed time:  0.20251250267028809
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Number nodes: 7
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Model prediction
Elapsed time:  0.061365604400634766
Loss:  8.304678916931152
start BP
end BP
Elapsed time:  0.27761030197143555
Number nodes: 8
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Model prediction
Elapsed time:  0.09138894081115723
Loss:  45.00788497924805
start BP
end BP
Elapsed time:  0.3689706325531006
Number nodes: 9
Create subgraph 

end BP
Elapsed time:  0.31750035285949707
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Number nodes: 8
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.09946727752685547
Loss:  7.753146171569824
start BP
end BP
Elapsed time:  0.4091200828552246
Number nodes: 9
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.12718653678894043
Loss:  74.55718994140625
start BP
end BP
Elapsed time:  0.5274488925933838
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Number nodes: 10
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.15747714042663574
Loss:  315.1643371582031
start BP
end BP
Elapsed time:  0.6659233570098877


Mean Loss Epoch 12:  49.16806221008301

 

Epoch: 13
Generate bat

...

Elapsed time:  0.20736932754516602
Loss:  10.886314392089844
start BP
end BP
Elapsed time:  0.7223641872406006


Mean Loss Epoch 585:  10.820532822608948

 

Epoch: 586
Generate batch of molecules
Classes allocation : (2, 1, 7)
Class with zero atoms:  None
Final counters:  [2, 1, 7]
Predict SOAP for all subgraphs from 1 to 10 atoms per subgraph
Number nodes: 1
Create subgraph input and label
[('O', 'O')]
Model prediction
Elapsed time:  0.01671457290649414
Loss:  15.841336250305176
start BP
end BP
Elapsed time:  0.03481864929199219
Number nodes: 2
Create subgraph input and label
[('O', 'O')]
Model prediction
Elapsed time:  0.011598348617553711
Loss:  60.11751174926758
start BP
end BP
Elapsed time:  0.03005218505859375
[('O', 'O')]
Number nodes: 3
Create subgraph input and label
[('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.022431135177612305
Loss:  21.84677505493164
start BP
end BP
Elapsed time:  0.0538945198059082
Number nodes: 4
Create subgraph input and label

Model prediction
Elapsed time:  0.010128021240234375
Loss:  14.709954261779785
start BP
end BP
Elapsed time:  0.040003061294555664
Number nodes: 2
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Model prediction
Elapsed time:  0.026760101318359375
Loss:  10.793481826782227
start BP
end BP
Elapsed time:  0.0446629524230957
Number nodes: 3
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Model prediction
Elapsed time:  0.034760475158691406
Loss:  16.365549087524414
start BP
end BP
Elapsed time:  0.06331229209899902
Number nodes: 4
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.04473423957824707
Loss:  12.268631935119629
start BP
end BP
Elapsed time:  0.10393404960632324
Number nodes: 5
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.0556948184967041
Loss:  12.46487712

Model prediction
Elapsed time:  0.04439568519592285
Loss:  9.199047088623047
start BP
end BP
Elapsed time:  0.0743870735168457
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Number nodes: 4
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.05160808563232422
Loss:  4.158766269683838
start BP
end BP
Elapsed time:  0.12024116516113281
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Number nodes: 5
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.05627131462097168
Loss:  1.8758411407470703
start BP
end BP
Elapsed time:  0.1627664566040039
Number nodes: 6
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.07479453086853027
Loss:  2.8061628341674805
start BP
end BP
Elapsed time: 

Model prediction
Elapsed time:  0.0610501766204834
Loss:  11.980525016784668
start BP
end BP
Elapsed time:  0.24181151390075684
Number nodes: 7
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.08872866630554199
Loss:  13.318424224853516
start BP
end BP
Elapsed time:  0.3207108974456787
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Number nodes: 8
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.11888647079467773
Loss:  3.266925811767578
start BP
end BP
Elapsed time:  0.4597785472869873
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Number nodes: 9
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('H', 'O'), ('C', 'C'), ('C', 'O'), ('O', 'O')]
Model prediction
Elapsed time:  0.16534781455993652
Loss:  2.5729146003723145
start BP
end BP
Elapsed time: 

end BP
Elapsed time:  0.35811901092529297
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Number nodes: 9
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Model prediction
Elapsed time:  0.09496712684631348
Loss:  3.3757004737854004
start BP
end BP
Elapsed time:  0.4584026336669922
Number nodes: 10
Create subgraph input and label
[('H', 'H'), ('H', 'C'), ('C', 'C')]
Model prediction
Elapsed time:  0.13588309288024902
Loss:  8.82224178314209
start BP
end BP
Elapsed time:  0.5820517539978027


Mean Loss Epoch 598:  24.480064821243285

 

Epoch: 599
Generate batch of molecules
Classes allocation : (6, 1, 3)
Class with zero atoms:  None
Final counters:  [6, 1, 3]
Predict SOAP for all subgraphs from 1 to 10 atoms per subgraph
Number nodes: 1
Create subgraph input and label
[('C', 'C')]
Model prediction
Elapsed time:  0.013153791427612305
Loss:  16.298656463623047
start BP
end BP
Elapsed time:  0.03253054618835449
[('C', 'C')]
Number nodes: 2
Create subgraph input and label
[('C', '

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (8, 8))
ax.plot(mean_loss_list)
plt.ylim(0, 100)